In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from functools import partial
from pathlib import Path
import torch
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt

alphas = (0.1, 0.25, 0.5, 0.75, 1.0)
ratios = (0.01, 0.1, 0.2, 0.5, 0.75)

metrics_names = ("train_acc", "val_acc_checkpoints", "train_top-5-acc", "val_top-5-acc_checkpoints", "train_nll", "val_nll_checkpoints")

def get_metrics(annot_quality, ratio, metrics_names):
    exp_name = "full_val_dino_ssl_linear_imagenet_1k"
    # exp_name = "test"
    exp_sub_name = f"annot_quality_{annot_quality}_ratio_{ratio}"
    dir_ = Path.cwd().parent / "stored" / exp_name / exp_sub_name
    return map(lambda name: (name, np.genfromtxt(dir_ / f"{name}.csv", delimiter=",", skip_header=1)), metrics_names)

ms = dict(get_metrics(0.1, 0.5, metrics_names))

train_loss = ms["train_nll"]
val_loss = ms["val_nll_checkpoints"]
train_acc = ms["train_acc"]
val_acc = ms["val_acc_checkpoints"]
train_acc_top_5 = ms["train_top-5-acc"]
val_acc_top_5 = ms["val_top-5-acc_checkpoints"]
fig, ax = plt.subplots()
#ax.plot(train_loss)
#ax.errorbar(x=val_loss[:, 0], y=val_loss[:, 2], yerr=val_loss[:, 3]);
ax.plot(train_acc)
ax.errorbar(x=val_acc[:, 0], y=val_acc[:, 2], yerr=val_acc[:, 3]);


In [ ]:
for a in alphas:
    for r in ratios:
        ms = dict(get_metrics(a, r, metrics_names))
        ext_ts, ts, mean, std = ms["val_top-5-acc_checkpoints"][-1]
        print(f"alpha: {a}, ratio: {r}, acc: {mean:.2f}+-{std:.2f}")

In [ ]:
R, A = np.meshgrid(ratios, alphas)
Z = torch.empty((len(ratios), len(alphas)))

for i_r, r in enumerate(ratios):
    for i_a, a in enumerate(alphas):
        ms = dict(get_metrics(a, r, metrics_names))
        _, _, mean, _ = ms["val_top-5-acc_checkpoints"][-1]
        Z[i_r, i_a] = mean
        
fig, ax = plt.subplots()
CS = ax.contour(R, A, Z)
ax.clabel(CS, inline=True, fontsize=10)
#plt.xscale('log')
ax.set_title
ax.set_xlabel("train set ratio")
ax.set_ylabel("annot qual. alpha")
from tikzplotlib import save as tikz_save
tikz_save("ssl_exp.tikz")